# OECD - Unemployment Rate and Inflation - International Comparisons

https://data-explorer.oecd.org/?lc=en

**Note**: GDP analysis moved to separate "FRED GDP International.ipynb" notebook due to OECD data limitations.

## Python setup

In [ ]:
# system imports
from io import StringIO
from pathlib import Path
from typing import Any, Sequence, cast
import time

In [ ]:
# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

In [ ]:
# local imports
import common
import mgplot as mg

In [ ]:
# plotting stuff
TARGET = {
    "ymin": 2,
    "ymax": 3,
    "color": "#dddddd",
    "label": "2-3% inflation target",
    "zorder": -1,
}
TARGET_V = TARGET | {"xmin": 2, "xmax": 3}
del TARGET_V["ymax"]
del TARGET_V["ymin"]

# Where to put the charts
CHART_DIR = "./CHARTS/OECD/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()

# display charts in notebook
SHOW = False

## Utility functions for OECD data capture

In [ ]:
location_map = {
    "AUS": "Australia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "CAN": "Canada",
    "CHL": "Chile",
    "CZE": "Czech Rep.",
    "DNK": "Denmark",
    "EST": "Estonia",
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "HUN": "Hungary",
    "ISL": "Iceland",
    "IRL": "Ireland",
    "ISR": "Israel",
    "ITA": "Italy",
    "JPN": "Japan",
    "KOR": "Korea",
    "LVA": "Latvia",
    "LUX": "Luxembourg",
    "MEX": "Mexico",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "NOR": "Norway",
    "POL": "Poland",
    "PRT": "Portugal",
    "SVK": "Slovak Rep.",
    "SVN": "Slovenia",
    "ESP": "Spain",
    "SWE": "Sweden",
    "CHE": "Switzerland",
    "TUR": "Türkiye",
    "GBR": "United Kingdom",
    "USA": "United States",
    "ARG": "Argentina",
    "BRA": "Brazil",
    "CHN": "China",
    "COL": "Colombia",
    "CRI": "Costa Rica",
    "IND": "India",
    "IDN": "Indonesia",
    "LTU": "Lithuania",
    "RUS": "Russia",
    "SAU": "Saudi Arabia",
    "ZAF": "South Africa",
    "ROU": "Romania",
    "BGR": "Bulgaria",
    "HRV": "Croatia",
}

In [ ]:
def get_oecd_table(
    agency: str, dataflow: str, filter_expr: str, options: str
) -> pd.DataFrame:
    """Capture a DataFrame from the OECD data API.
    OECD updated data API:
        https://sdmx.oecd.org/public/rest/data/
        <agency identifier>,<dataflow identifier>,<dataflow version>/
        <filter expression>[?<optional parameters>]

    Use https://data-explorer.oecd.org/?lc=en
    to get the necessary identifiers."""

    stem = "https://sdmx.oecd.org/public/rest/data"
    options = options + "&format=csv"
    options = options[1:] if options[0] == "&" else options

    url = f"{stem}/{agency},{dataflow}/{filter_expr}?{options}"
    contents = common.request_get(url).decode("utf-8")
    df = pd.read_csv(StringIO(contents))
    pvt = df.pivot(index="TIME_PERIOD", columns="REF_AREA", values="OBS_VALUE")

    pvt = pvt.dropna(how="all", axis=1)
    pvt = pvt.dropna(how="all", axis=0)

    return pvt


SOURCE = "OECD Data Explorer"

In [ ]:
def check_missing(df: pd.DataFrame) -> None:
    """Check data downloaded from OECD for missing columns."""

    # external check:
    missing = list(set(location_map.keys()) - set(df.columns))
    if missing:
        text = ", ".join([location_map[x] for x in missing])
        print(f"Missing national data for {text}")

    # internal check
    final_row = df.iloc[-1]
    missing_count = final_row.isna().sum()
    if missing_count:
        print(f"Final period: {final_row.name}")
        print(f"Missing data count for final period: {missing_count}")
        print(f"Missing data belongs to: {df.columns[final_row.isna()].to_list()}")
        print(f"Nations with final data: {df.columns[final_row.notna()].to_list()}")

In [ ]:
def remove_non_national(df: pd.DataFrame) -> pd.DataFrame:
    """Remove non-national columns."""

    remove = df.columns.difference(pd.Index(location_map.keys()))
    if len(remove):
        print(f"Removing columns: {remove}")
        df = df.drop(remove, axis=1)
    return df

In [ ]:
def combine(left: None | pd.DataFrame, right: pd.DataFrame) -> pd.DataFrame:
    """Concatenate two DataFrames horizontally.
    Ignore columns in right that we already have in left.
    Therefore you need to capture the most relevant dataflows first.
    Used when we need combine get data from multiple OECD tables."""

    if left is None:
        left = right
    else:
        duplicates = left.columns.intersection(right.columns)
        if len(duplicates):
            right = right.drop(duplicates, axis=1)
        left = pd.concat([left, right], axis=1)

    return left

In [ ]:
def fix_monthly(df: pd.DataFrame) -> pd.DataFrame:
    """Fix quarterly data as monthly. Note, where the OECD places
    quarterly data in a monthly series, it places this data in the
    mid-quarter month. So we will replicate with the conversion of
    quarterly fsts to monthly."""

    df.index += 1  # mid-period
    index = pd.period_range(start=df.index.min(), end=df.index.max())
    df = df.reindex(index)
    df = df.interpolate(limit_area="inside", limit=2, axis=0)
    return df

In [ ]:
WEB_DELAY = 2  # seconds

## Utility functions for plotting

In [ ]:
def get_chart_groups() -> dict[str, list[str]]:
    """Get chart groups, with no more than 6 lines
    per chart."""

    of_interest = ["AUS", "USA", "CAN", "DEU", "GBR", "JPN"]
    anglosphere = ["AUS", "USA", "CAN", "NZL", "GBR", "IRL"]
    major_europe = ["FRA", "DEU", "ITA", "GBR", "RUS", "ESP"]
    largest_economies = ["USA", "CHN", "JPN", "DEU", "GBR", "IND"]
    asia = ["KOR", "JPN", "CHN", "IND", "IDN"]
    north_europe = ["DNK", "SWE", "NOR", "ISL", "FIN", "GBR"]
    baltic_europe = ["LVA", "LTU", "EST"]
    central_europe = ["CZE", "HUN", "SVK", "SVN", "POL", "GRC"]
    west_europe = ["BEL", "ESP", "PRT", "NLD", "LUX", "FRA"]
    italo_germanic_europe = ["DEU", "AUT", "CHE", "ITA"]
    n_america = ["USA", "CAN", "MEX"]
    c_s_america = ["CHL", "BRA", "COL", "CRI"]
    other = ["AUS", "NZL", "SAU", "ZAF", "ISR"]
    high_inflation = [
        "TUR",
        "ARG",
    ]

    charts = {
        "of_interest": of_interest,
        "anglosphere": anglosphere,
        "major_europe": major_europe,
        "largest_economies": largest_economies,
        "asia": asia,
        "north_europe": north_europe,
        "baltic_europe": baltic_europe,
        "central_europe": central_europe,
        "west_europe": west_europe,
        "italo_germanic_europe": italo_germanic_europe,
        "n_america": n_america,
        "c_s_america": c_s_america,
        "high_inflation": high_inflation,
        "other": other,
    }

    return charts


chart_sets = get_chart_groups()

In [ ]:
PW_COUNTER = 0  # global filename suffix - I know, ugly.

In [ ]:
MEAN_MEDIAN = 0.80  # proportion of non-na data points to plot mean and median


def plot_world(
    data: pd.DataFrame,
    exclusions: None | list[str] = None,
    **kwargs: Any,
) -> None:
    """Plot Australia vs the OECD monitored mean and median."""

    # Exclude problematic OECD states
    data = data.rename(columns=location_map)
    if exclusions is not None:
        for c in exclusions:
            if c in data.columns:
                data = data.drop(c, axis=1)

    # plot remaining OECD states without legend label using the _ trick
    mapper = {x: f"_{x}" for x in data.columns}
    data = data.rename(columns=mapper)
    ax = mg.line_plot(data, width=0.3, color='blue')
    back = {y: x for x, y in mapper.items()}
    data = data.rename(columns=back)

    # plot mean if THRESHOLD proportion of non-na data points met
    mean = data.mean(axis=1).where(
        data.notna().sum(axis=1) >= len(data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    mean.name = "OECD monitored mean"
    median = data.median(axis=1).where(
        data.notna().sum(axis=1) >= len(data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    median.name = "OECD monitored median"
    mg.line_plot(
        mean, ax=ax, color="darkblue", style="--", width=2, label_series=True
    )
    mg.line_plot(
        median, ax=ax, color="darkred", style=":", width=2, label_series=True
    )

    # plot
    mg.line_plot(
        data["Australia"].dropna(),
        ax=ax,
        color="darkorange",
        width=3,
        label_series=True,
    )
    global PW_COUNTER  # yes, this is ugly
    PW_COUNTER = PW_COUNTER + 1
    mg.finalise_plot(
        ax,
        xlabel=None,
        y0=True,
        rfooter=SOURCE,
        tag=str(PW_COUNTER),
        legend={"loc": "best", "fontsize": "xx-small"},
        **kwargs,
        show=SHOW,
    )

In [ ]:
def plot_annual(data: pd.DataFrame, **kwargs: Any) -> None:
    """Quick Annual Charts, based on chart_sets from above."""

    for tag, chart_set in chart_sets.items():
        chart_set = sorted(set(chart_set).intersection(set(data.columns)))
        cs = data[chart_set].rename(columns=location_map)
        mg.line_plot_finalise(
            cs,
            tag=tag,
            xlabel=None,
            dropna=True,
            y0=True,
            width=2,
            rfooter=SOURCE,
            **kwargs,
            show=SHOW,
        )

#### OHLC plotting

In [ ]:
OHLC_DEFAULT_N = 13  # months

In [ ]:
def get_recent_ohlc(
    data: pd.DataFrame,
    n: int = OHLC_DEFAULT_N,  # months
    exclude: Sequence = (),  # excluded nations
) -> pd.DataFrame:
    """For a dataset, build a table of Open, Highm Low, Close
    points for last valid n months in each column."""

    # compare progress over 13 (12 + 1) months because Australia
    # and New Zealand only collect CPI measures quarterly
    index = ["Open", "High", "Low", "Close"]
    summary = pd.DataFrame([], index=index)  # return vehicle
    for name in data.columns:
        if name in exclude:
            continue
        column = data[name]
        last_valid = cast(pd.Period, column.last_valid_index())  # mypy cast
        time_period = pd.period_range(end=last_valid, periods=n)
        if time_period.min() < column.index.min():
            continue
        frame = column[time_period]
        open_ = frame.iloc[0]
        high = frame.max()
        low = frame.min()
        close = frame.iloc[-1]
        key = f"{name} {str(last_valid.year)[2:]}-{last_valid.month:02d}"
        summary[key] = pd.Series([open_, high, low, close], index=index)
    summary = summary.T.sort_values("Close")
    return summary

In [ ]:
def plot_ohlc(
    ohlc_df: pd.DataFrame,
    horizontal: bool = True,  # horizontal v vertical bars
    n: int = OHLC_DEFAULT_N,  # months
    **kwargs: Any,
) -> None:
    """Plot data in ohlc_df in a open-high-low-close style."""

    def xy(x, y):
        return (x, y) if horizontal else (y, x)

    def set_limits(ax: plt.Axes) -> None:
        minimum = min(0, ohlc_df["Low"].min())  # include zero
        maximum = ohlc_df["High"].max()
        adjustment = (maximum - minimum) * 0.025
        limits = minimum - adjustment, maximum + adjustment
        if horizontal:
            ax.set_xlim(*limits)
        else:
            ax.set_ylim(*limits)

    # default settings
    kwargs["lfooter"] = kwargs.get(
        "lfooter",
        "Year and month of latest print in the axis labels. "
        f"Range is the {n} months up to and including the latest data. ",
    )
    kwargs["rfooter"] = kwargs.get("rfooter", SOURCE)
    kwargs["legend"] = kwargs.get("legend", {"loc": "best", "fontsize": "6"})
    kwargs["y0"] = kwargs.get("y0", not horizontal)
    kwargs["x0"] = kwargs.get("x0", horizontal)

    # canvass
    _, ax = plt.subplots()

    # sort out chart orientation
    good, bad = "darkblue", "darkorange"  # for colour blindness
    bar_method = ax.barh if horizontal else ax.bar
    reference: str = "left" if horizontal else "bottom"
    range_ = ohlc_df["High"] - ohlc_df["Low"]
    open_marker = "^" if horizontal else "<"
    close_marker = "v" if horizontal else ">"
    color = [
        good if open > close else bad
        for open, close in zip(ohlc_df.Open, ohlc_df.Close)
    ]

    # plot
    base: dict[str, np.ndarray] = {reference: ohlc_df["Low"].to_numpy()}
    bar_method(  # type: ignore[arg-type]
        ohlc_df.index,
        range_,
        color=color,
        linewidth=1.0,
        edgecolor="black",
        label=f"Range of prints through the {n} months",
        alpha=0.15,
        **base,  # type: ignore[arg-type]
    )
    ax.plot(
        *xy(ohlc_df["Open"], ohlc_df.index),
        marker=open_marker,
        linestyle="None",
        label=f"First print in the {n} months",
        color=good,
        markersize=5,
    )
    ax.plot(
        *xy(ohlc_df["Close"], ohlc_df.index),
        marker=close_marker,
        linestyle="None",
        label=f"Last print in the {n} months",
        color=bad,
        markersize=5,
    )
    ax.tick_params(axis="both", which="major", labelsize="xx-small")
    set_limits(ax=ax)
    if not horizontal:
        ax.set_xticklabels(ohlc_df.index, rotation=90)
    mg.finalise_plot(ax, **kwargs)

## Unemployment rate

In [ ]:
def get_ue_data():
    """Get OECD unemployment rate data.
    Challenge: NZL and CHE only reported quarterly."""

    agency = "OECD.SDD.TPS"
    dataflow = "DSD_LFS@DF_IALFS_UNE_M,1.0"
    filter_exprs = (
        "..._Z.Y._T.Y_GE15..M",  # get monthly data first
        "..._Z.Y._T.Y_GE15..Q",  # then get quarterly
    )
    options = "startPeriod=2000-01"

    combined = None
    for filter_expr in filter_exprs:
        ue = get_oecd_table(agency, dataflow, filter_expr, options)
        ue.index = pd.PeriodIndex(ue.index, freq="M")
        if filter_expr[-1] == "Q":
            ue = fix_monthly(ue)
        combined = combine(combined, ue)
        time.sleep(WEB_DELAY)  # be nice to the OECD server

    return remove_non_national(combined)


ue_rates = get_ue_data()

In [ ]:
check_missing(ue_rates)

In [ ]:
def plot_ue(data: pd.DataFrame) -> None:
    """Plot unemployment rate data."""

    kwargs: dict[str, Any] = {
        "title": "Unemployment rates",
        "ylabel": "Per cent",
        "legend": {"loc": "best", "fontsize": "x-small"},
    }
    plot_annual(data, **kwargs)


plot_ue(ue_rates[ue_rates.index.year >= 2019])

In [ ]:
def plot_world_ue() -> None:
    "Plot comparative unemployment rates."

    kwargs: dict[str, Any] = {
        "title": "Australian unemployment rate in the world context",
        "ylabel": "Per cent",
        "lfooter": "OECD monitored nations. Mean and median calculated where "
        f"{int(MEAN_MEDIAN*100)}% or more nations report.",
    }
    plot_world(ue_rates[ue_rates.index.year >= 2017], **kwargs)


plot_world_ue()

In [ ]:
ue_rates.tail()

In [ ]:
def plot_ohlc_ue() -> None:
    """Plot recent unemployment rate data in an OHLC style."""

    for n in [13, 25, 37, 49, 61]:
        ue_ohlc = get_recent_ohlc(ue_rates.rename(columns=location_map), n=n)
        plot_ohlc(
            ue_ohlc,
            horizontal=False,
            n=n,
            title=f"OECD Unemployment Rates - previous {n} months",
            ylabel="Per cent",
            show=SHOW,
        )


plot_ohlc_ue()

## Inflation 

In [ ]:
EXCLUDE = ["Türkiye", "Russia", "Argentina"]
# Turkey and Argentina have rampant inflation
# Russia not updating data during war

In [ ]:
def get_annual_inflation() -> pd.DataFrame:
    """Get OECD Annual Inflation Data.

    The challenges:
    - two different dataflows,
    - while most nations report monthly, some report quarterly, and
    - Australia transitioned from quarterly to monthly CPI in April 2025,
      but the COICOP2018 dataflow still treats it as quarterly.
      The older dataflow has the monthly data.
    - New Zealand still reports quarterly."""

    agency = "OECD.SDD.TPS"
    dataflows = (
        "DSD_PRICES_COICOP2018@DF_PRICES_C2018_ALL,1.0",  # must be first
        "DSD_PRICES@DF_PRICES_ALL,1.0",
    )
    filter_exprs = (
        ".M.N.CPI.PA._T.N.GY",  # Monthly must be first
        ".Q.N.CPI.PA._T.N.GY",
    )

    options = "startPeriod=2019-05"

    combined = None
    for dataflow in dataflows:
        for filter_expr in filter_exprs:
            pvt = get_oecd_table(agency, dataflow, filter_expr, options)
            pvt.index = pd.PeriodIndex(pvt.index, freq="M")
            if filter_expr[1] == "Q":
                pvt = fix_monthly(pvt)
            combined = combine(combined, pvt)
            time.sleep(WEB_DELAY)  # just to be nice to the server.

    combined = remove_non_national(cast(pd.DataFrame, combined))

    # Patch: Australia transitioned to monthly CPI from April 2025.
    # The older dataflow (DSD_PRICES) has this monthly data, but combine()
    # skipped it because AUS already existed from COICOP2018 (quarterly).
    # Fetch it separately and overwrite.
    try:
        aus_monthly = get_oecd_table(
            agency,
            "DSD_PRICES@DF_PRICES_ALL,1.0",
            "AUS.M.N.CPI.PA._T.N.GY",
            options,
        )
        if not aus_monthly.empty and "AUS" in aus_monthly.columns:
            aus_monthly.index = pd.PeriodIndex(aus_monthly.index, freq="M")
            # Overwrite AUS with true monthly data where available
            combined.loc[aus_monthly.index, "AUS"] = aus_monthly["AUS"]
            print(f"Patched AUS with monthly data: {aus_monthly.index.min()} to {aus_monthly.index.max()}")
    except Exception as e:
        print(f"Warning: Could not fetch AUS monthly data: {e}")

    return combined


annual_inflation = get_annual_inflation()

# Move quarterly data to the end of the quarter.
# The OECD places quarterly data in the mid-quarter month.
# AUS quarterly portion: only shift periods before the monthly transition.
aus_monthly_start = pd.Period("2025-04", freq="M")
aus_quarterly = annual_inflation["AUS"].loc[annual_inflation.index < aus_monthly_start]
annual_inflation.loc[annual_inflation.index < aus_monthly_start, "AUS"] = aus_quarterly.shift(1)
# NZL still reports quarterly
annual_inflation["NZL"] = annual_inflation["NZL"].shift(1)

display(annual_inflation[["AUS", "NZL"]].tail(12))

In [ ]:
check_missing(annual_inflation)

In [ ]:
def plot_ohlc_inf() -> None:
    """Plot recent unemployment rate data in an OHLC style."""

    for n in [13, 25, 37, 49]:
        ohlc = get_recent_ohlc(
            annual_inflation.rename(columns=location_map), n=n, exclude=EXCLUDE
        )
        plot_ohlc(
            ohlc,
            horizontal=False,
            n=n,
            title=f"OECD Annual Inflation Rates - previous {n} months",
            ylabel="Per cent",
            axhspan=TARGET,
            show=SHOW,
        )


plot_ohlc_inf()

In [ ]:
def plot_world_inflation():
    "Plot World Inflation."

    kwargs = {
        "title": "Australian inflation in the world context",
        "ylabel": "Per cent per year",
        "lfooter": f'OECD monitored excluding: {", ".join(EXCLUDE)}. '
        f"Mean/median calculated when >{int(MEAN_MEDIAN*100)}% of nations report.",
        "axhspan": TARGET,
    }

    plot_world(
        annual_inflation,
        exclusions=EXCLUDE,
        **kwargs,
    )


plot_world_inflation()

In [ ]:
def plot_annual_inflation(data: pd.DataFrame):
    "Plot annual inflation."

    kwargs = {
        "title": "Annual Consumer Price Inflation",
        "ylabel": "Per cent per Year",
        "axhspan": TARGET,
    }
    plot_annual(data, **kwargs)


plot_annual_inflation(annual_inflation)

## Population (indexed to 2000 = 100)

In [ ]:
def get_population_data() -> pd.DataFrame:
    """Get OECD total population data (annual, both sexes, all ages)."""

    agency = "OECD.ELS.SAE"
    dataflow = "DSD_POPULATION@DF_POP_HIST,1.0"
    filter_expr = "..PS._T._T."
    options = "startPeriod=2000"

    pop = get_oecd_table(agency, dataflow, filter_expr, options)
    pop.index = pd.PeriodIndex(pop.index, freq="Y")
    pop = remove_non_national(pop)
    return pop


population = get_population_data()
check_missing(population)
display(population.tail())

In [37]:
# Index population to base year = 100
START_YEARS = [2000, 2010, 2022]

In [38]:
def plot_world_population(start_year: int) -> None:
    """Plot Australia's population growth vs OECD nations, with endpoint labels."""

    base = pd.Period(str(start_year), freq="Y")
    pop_indexed = population.loc[base:].div(population.loc[base]) * 100
    pop_indexed = pop_indexed.dropna(how="all", axis=1)
    data = pop_indexed.rename(columns=location_map)

    # plot all nations as thin blue lines (hidden from legend)
    mapper = {x: f"_{x}" for x in data.columns}
    data_hidden = data.rename(columns=mapper)
    ax = mg.line_plot(data_hidden, width=0.3, color='blue')
    back = {y: x for x, y in mapper.items()}
    data_hidden = data_hidden.rename(columns=back)

    # mean and median with endpoint annotations
    mean = data.mean(axis=1).where(
        data.notna().sum(axis=1) >= len(data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    mean.name = "OECD monitored mean"
    median = data.median(axis=1).where(
        data.notna().sum(axis=1) >= len(data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    median.name = "OECD monitored median"
    mg.line_plot(
        mean, ax=ax, color="darkblue", style="--", width=2,
        label_series=True, annotate=True,
    )
    mg.line_plot(
        median, ax=ax, color="darkred", style=":", width=2,
        label_series=True, annotate=True,
    )

    # Australia with endpoint annotation
    aus = data["Australia"].dropna()
    mg.line_plot(
        aus, ax=ax, color="darkorange", width=3,
        label_series=True, annotate=True,
    )

    # Name-annotate nations with higher endpoint than Australia
    aus_end = aus.iloc[-1]
    for country in data.columns:
        if country == "Australia":
            continue
        series = data[country].dropna()
        if len(series) == 0:
            continue
        end_val = series.iloc[-1]
        if end_val > aus_end:
            ax.annotate(
                country,
                xy=(series.index[-1].ordinal, end_val),
                fontsize=6,
                color="black",
                ha="left",
                va="center",
                xytext=(5, 0),
                textcoords="offset points",
            )

    global PW_COUNTER
    PW_COUNTER = PW_COUNTER + 1
    mg.finalise_plot(
        ax,
        title=f"Australian population growth in the world context since {start_year}",
        ylabel=f"Index ({start_year} = 100)",
        xlabel=None,
        y0=True,
        rfooter=SOURCE,
        lfooter="OECD monitored nations. Mean and median calculated where "
        f"{int(MEAN_MEDIAN*100)}% or more nations report.",
        tag=str(PW_COUNTER),
        legend={"loc": "best", "fontsize": "xx-small"},
        show=SHOW,
    )


for yr in START_YEARS:
    plot_world_population(yr)

In [39]:
def plot_population_bar(start_year: int) -> None:
    """Bar chart of annualised population growth rates, sorted."""

    base = pd.Period(str(start_year), freq="Y")
    pop_indexed = population.loc[base:].div(population.loc[base]) * 100
    pop_indexed = pop_indexed.dropna(how="all", axis=1)

    # Get each nation's last valid value and compute annualised growth
    annualised = {}
    for col in pop_indexed.columns:
        series = pop_indexed[col].dropna()
        if len(series) == 0:
            continue
        last_val = series.iloc[-1]
        last_year = series.index[-1]
        n_years = last_year.year - start_year
        if n_years <= 0:
            continue
        annualised[col] = ((last_val / 100) ** (1 / n_years) - 1) * 100

    annualised = pd.Series(annualised).rename(index=location_map).sort_values()

    ax = mg.bar_plot(annualised, stacked=True)
    ax.tick_params(axis="x", labelrotation=90)
    mg.finalise_plot(
        ax,
        title=f"Annualised population growth since {start_year}",
        ylabel="Per cent per year",
        rfooter=SOURCE,
        lfooter="OECD monitored nations. Compound annual growth rate.",
        show=SHOW,
    )


for yr in START_YEARS:
    plot_population_bar(yr)

## Finished

In [40]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2026-02-23 20:34:08

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.9.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

IPython   : 9.9.0
matplotlib: 3.10.8
mgplot    : 0.2.18
numpy     : 2.4.0
pandas    : 2.3.3
pathlib   : 1.0.1
typing    : 3.10.0.0

Watermark: 2.6.0

